In [ ]:
import json

import pandas as pd

import gudhi as gd
import gudhi.wasserstein as wasserstein
import gudhi.hera as hera

from collections import defaultdict
from tqdm import tqdm

from itertools import combinations, combinations_with_replacement, product

import ripserplusplus as rpp

from scipy.spatial import distance_matrix
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import numpy as np
import torch
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
cmap = 'viridis'

# Visualization

### Many models - many datasets

In [ ]:
# datasets = [
#     'Synthetic/Circle', 
#     'Synthetic/2Clusters', 
#     'Synthetic/3Clusters', 
# # #     'Synthetic/Infty', 
#     'Synthetic/RandomCube'
# ]

datasets = [
    'MNIST',
    'F-MNIST', 
    'COIL-20',
    'scRNA_mice',
    'scRNA_melanoma'
#     'c.elegans'
]

models = {
    'umap':'UMAP',
    'tsne':'t-SNE',
    'pacmap':'PaCMAP',
    'phate':'PHATE',
    'ivis':'Ivis',
    'Basic AutoEncoder':'AE',
    'Topological AutoEncoder':'TopoAE (Moor et.al.)',
    'RTD AutoEncoder H1':'RTD'
}
dataset_names = {'RandomCube':'Random', '2Clusters':'2 Clusters', '3Clusters':'3 Clusters'}
#'scRNA melanoma':'scRNA melanoma', 'scRNA_mice':'scRNA mice', 
versions = {
    'scRNA_melanoma':'d2', 
    'scRNA_mice':'d2',
    'COIL-20':'d2', 
    'F-MNIST':'d2', 
    'MNIST':'d2'}
add_original = False

In [ ]:
def plot_2d(latent, labels, alpha=0.7, title="", fontsize=25, s=2.0, ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    scatter = ax.scatter(latent[:, 0], latent[:, 1], alpha=alpha, c=labels, s=s, label=labels)
#     legend = ax.legend(*scatter.legend_elements(num=len(np.unique(labels))), loc="upper left", title="Types")
#     ax.add_artist(legend)
    if len(title):
        ax.set_title(title, fontsize=fontsize)
    return ax

In [ ]:
# plot multiple datasets latent representations
fig, axes = plt.subplots(len(datasets), len(models)+int(add_original), figsize=((len(models)+int(add_original))*6, len(datasets)*6), squeeze=False)
for i, dataset in enumerate(datasets):
    version = versions.get(dataset, "")
    print(f"dataset: {dataset}, version: {version}")
    labels = None # refactor
    try:
        labels = np.load(f"data/{dataset}/prepared/train_labels.npy")
    except FileNotFoundError:
        pass
    try:
        labels = np.load(f"data/{dataset}/prepared/labels.npy")
    except FileNotFoundError:
        pass
    if add_original:
        original_data = np.load(f"data/{dataset}/prepared/train_data.npy")
        axes[i][0].scatter(original_data[:, 0], original_data[:, 1], c=labels, s=20.0, alpha=0.7, cmap=plt.cm.get_cmap('nipy_spectral', 11))
        if i == 0:
            axes[0][0].set_title('Original data', fontsize=40)
        axes[i][0].tick_params(
            axis='both', 
            which='both', 
            bottom=False, 
            top=False,
            labelbottom=False,
            right=False,
            left=False,
            labelleft=False
        )
        d = dataset.split('/')[-1]
        d = dataset_names.get(d, d)
        axes[i][0].set_ylabel(d, fontsize=40)
    for j, name in enumerate(models):
        if add_original:
            j+=1
            
        latent = None
        
        potential_filenames = [
            f'data/{dataset}/{name}_output_{version}.npy', 
            f'data/{dataset}/{name}_output_d2.npy', 
            f'data/{dataset}/{name}_output_.npy',
            f'data/{dataset}/{name}_output.npy'
        ]
        for n in potential_filenames:
            try:
                latent = np.load(n)
                break
            except FileNotFoundError:
                print("")
        if latent is None:
            raise FileNotFoundError(f'No file for model: {name}, dataset: {dataset}')
        axes[i][j].scatter(latent[:, 0], latent[:, 1], c=labels, s=20.0, alpha=0.7, cmap=plt.cm.get_cmap('nipy_spectral', 11))
        if i == 0:
            axes[i][j].set_title(f'{models[name]}', fontsize=40)
        if j == 0 and not add_original:
            d = dataset.split('/')[-1]
            d = dataset_names.get(d, d)
            axes[i][j].set_ylabel(d, fontsize=40)
        axes[i][j].tick_params(
            axis='both', 
            which='both', 
            bottom=False, 
            top=False,
            labelbottom=False,
            right=False,
            left=False,
            labelleft=False
        )
plt.savefig('results/real.png')

### Many datasets - original data

In [ ]:
# original data
datasets = [
    'Synthetic/Circle', 
    'Synthetic/2Clusters', 
    'Synthetic/3Clusters', 
    'Synthetic/RandomCube'
]

fig, axes = plt.subplots(1, len(datasets), figsize=(len(datasets)*6, 6*1))

for i, dataset in enumerate(datasets):
    labels = None # refactor
    try:
        labels = np.load(f"data/{dataset}/prepared/train_labels.npy")
    except FileNotFoundError:
        pass
    try:
        labels = np.load(f"data/{dataset}/prepared/labels.npy")
    except FileNotFoundError:
        pass
    try:
        data = np.load(f'data/{dataset}/prepared/train_data.npy')
    except FileNotFoundError:
        data = np.load(f'data/{dataset}/prepared/data.npy')
    name = dataset.split('/')[-1]
    axes[i].scatter(data[:, 0], data[:, 1], c=labels, s=60.0, alpha=0.7, cmap=plt.cm.get_cmap('nipy_spectral', 11))
    axes[i].set_title(name, fontsize=30)
    axes[i].tick_params(
        axis='both', 
        which='both', 
        bottom=False, 
        top=False,
        labelbottom=False,
        right=False,
        left=False,
        labelleft=False
    )

### One dataset - many models

In [ ]:
dataset_name = 'COIL-20'

models = [
#     'umap':'UMAP',
#     'tsne':'t-SNE',
#     'Basic AutoEncoder':'AE',
#     'Topological AutoEncoder':'TopoAE (Moor et.al.)',
#     ('umap', '', 'UMAP'),
#     ('tsne', '', 't-SNE'),
    ('RTD AutoEncoder H1', 'geodesic', 'RTD-AE g'),
    ('RTD AutoEncoder H1 min_max', '3d', 'RTD-AE MM'),
]

In [ ]:
fig, axes = plt.subplots(1, len(models), figsize=(len(models)*6, 6))
for j, m in enumerate(models):
    name, version, print_name = m
#     version = versions.get(name, "")
    latent = np.load(f'data/{dataset_name}/{name}_output_{version}.npy')
    try:
        labels = np.load(f'data/{dataset_name}/{name}_labels_{version}.npy')
    except FileNotFoundError:
        labels = np.ones(latent.shape[0])
    if latent.shape[1] > 2:
        print(f"Error: {name}")
    axes[j].scatter(latent[:, 0], latent[:, 1], c=labels, s=20.0, alpha=0.7, cmap=plt.cm.get_cmap('nipy_spectral', 11))
    axes[j].set_title(f'{print_name}', fontsize=30)
    axes[j].tick_params(
        axis='both', 
        which='both', 
        bottom=False, 
        top=False,
        labelbottom=False,
        right=False,
        left=False,
        labelleft=False
    )

# Metrics

In [ ]:
dataset_name = 'COIL-20'
version = 'd16'

models = {
#     'pca':'PCA',
#     'mds':'MDS',
    'phate':'PHATE',
#     'pacmap':'PaCMAP',
#     'ivis':'Ivis',
#     'umap':'UMAP',
#     'tsne':'t-SNE',
#     'Basic AutoEncoder':'AE',
#     'Topological AutoEncoder':'TopoAE (Moor et.al.)',
#     'RTD AutoEncoder H1':'RTD-AE'
}

## Calculate distance matrix

In [ ]:
import torch

def pdist_gpu(a, b, device = 'cuda:1'):
    A = torch.tensor(a, dtype = torch.float64)
    B = torch.tensor(b, dtype = torch.float64)

    size = (A.shape[0] + B.shape[0]) * A.shape[1] / 1e9
    max_size = 0.2

    if size > max_size:
        parts = int(size / max_size) + 1
    else:
        parts = 1

    pdist = np.zeros((A.shape[0], B.shape[0]))
    At = A.to(device)

    for p in range(parts):
        i1 = int(p * B.shape[0] / parts)
        i2 = int((p + 1) * B.shape[0] / parts)
        i2 = min(i2, B.shape[0])

        Bt = B[i1:i2].to(device)
        pt = torch.cdist(At, Bt)
        pdist[:, i1:i2] = pt.cpu()

        del Bt, pt
        torch.cuda.empty_cache()

    del At

    return pdist

def zero_out_diagonal(distances):# make 0 on diagonal
    return distances * (np.ones_like(distances) - np.eye(*distances.shape))

In [ ]:
data = np.load(f'data/{dataset_name}/prepared/train_data.npy')
data = data.reshape(data.shape[0], -1)
ids = np.random.choice(np.arange(len(data)), size=min(30000, len(data)), replace=False)
data = data[ids]

In [ ]:
original_distances = pdist_gpu(data, data)

In [ ]:
original_distances = zero_out_diagonal(original_distances)

## Pearson correlation for pairwise distances

In [ ]:
try:
    labels = np.load(f'data/{dataset_name}/prepared/train_labels.npy')
except FileNotFoundError:
    labels = np.load(f'data/{dataset_name}/prepared/train_data.npy')
# ids = np.random.choice(np.arange(0, len(labels)), size=min(6000, len(labels)), replace=False)

def get_distances(data):
    data = data.reshape(data.shape[0], -1)
    distances = distance_matrix(data, data)
    distances = distances[np.triu(np.ones_like(distances), k=1) > 0]
    return distances
 # take only different 

In [ ]:
labels.shape

In [ ]:
results = {}
for model_name in models:
    try:
        latent = np.load(f'data/{dataset_name}/{model_name}_output_{version}.npy')[ids]
        latent_distances = pdist_gpu(latent, latent)
    except FileNotFoundError:
        continue
    results[model_name] = pearsonr(
        latent_distances[np.triu(np.ones_like(original_distances), k=1) > 0], 
        original_distances[np.triu(np.ones_like(original_distances), k=1) > 0])[0]

In [ ]:
results

In [ ]:
latent = np.load(f'data/{dataset_name}/{model_name}_output_{version}.npy')
x = latent[:, 0]
y = latent[:, 1]
plt.scatter(x, y)

## Accuracy

In [ ]:
version

In [ ]:
train_labels = np.load(f"data/{dataset_name}/prepared/train_labels.npy")
is_there_test = True
try:
    test_labels = np.load(f"data/{dataset_name}/prepared/test_labels.npy")
except FileNotFoundError:
    is_there_test = False
    train_ids, test_ids = train_test_split(np.arange(0, len(train_labels)), test_size=0.2)
    test_labels = train_labels[test_ids]
    train_labels = train_labels[train_ids]
    
results = defaultdict(dict)

for model_name in models:
    try:
        train_data = np.load(f'data/{dataset_name}/{model_name}_output_{version}.npy')
    except FileNotFoundError:
        continue
    if is_there_test:
        test_data = np.load(f'data/{dataset_name}/{model_name}_output_{version}_test.npy')
    else:
        test_data = train_data[test_ids]
        train_data = train_data[train_ids]
#         print('test_data not found')
#     for k in [3, 5, 10, 50, 100]:
#         classifier = KNeighborsClassifier(n_neighbors=k)
#         classifier.fit(train_data, train_labels)
#         results[model_name][f'knn_k{k}'] = accuracy_score(test_labels, classifier.predict(test_data))
    C = 1.0
    classifier = SVC(C=C, kernel='rbf')
    classifier.fit(train_data, train_labels)
    results[model_name][f'svm_C{C}'] = accuracy_score(test_labels, classifier.predict(test_data))

In [ ]:
results

In [ ]:
pd.DataFrame.from_dict(results)

## Wasserstein

In [ ]:
def calculate_persistence_gd(distances, dim=3):
    skeleton = gd.RipsComplex(distance_matrix = distances)
    simplex_tree = skeleton.create_simplex_tree(max_dimension=dim)
    barcodes = simplex_tree.persistence()
    pbarcodes = {}
    for i in range(dim+1):
        pbarcodes[i] = [[b[1][0], b[1][1]] for b in barcodes if b[0] == i]
    return pbarcodes


def cast_to_normal_array(barcodes):
    return np.array([[b, d] for b, d in barcodes])

In [ ]:
n_runs = 5
batch_size = 2048

data = np.load(f'data/{dataset_name}/prepared/train_data.npy')
data = data.reshape(len(data), -1)

if batch_size > len(data):
    n_runs=1
    
max_dim = 1
results = defaultdict(dict)

for i in range(n_runs):
    ids = np.random.choice(np.arange(0, len(original_distances)), size=min(batch_size, len(original_distances)), replace=False)
    
    x = data[ids]
#     distances = distance_matrix(x, x)
    distances = original_distances[ids][:, ids]/np.percentile(original_distances.flatten(), 90)
    barcodes = {'original':rpp.run(f'--format distance --dim {max_dim}', data=distances)}
    datasets = {'original':x}
    
    for model_name in tqdm(models):
        try:
            x = np.load(f'data/{dataset_name}/{model_name}_output_{version}.npy')
            x = x.reshape(len(x), -1)[ids]
            distances = zero_out_diagonal(pdist_gpu(x, x))
            distances = distances/np.percentile(distances.flatten(), 90)
            if not np.isnan(distances).any():
                barcodes[model_name] = rpp.run(f'--format distance --dim {max_dim}', data=distances)
            datasets[model_name] = x
        except FileNotFoundError:
            continue
        for dim in range(max_dim+1):
            if model_name not in results[dim]:
                results[dim][model_name] = []
            results[dim][model_name].append(hera.wasserstein_distance(
                cast_to_normal_array(barcodes['original'][dim]), 
                cast_to_normal_array(barcodes[model_name][dim]),
                internal_p=1.0
            ))

In [ ]:
for dim in range(max_dim+1):
    print(f'Dimension: {dim}')
    for model_name in results[dim]:
        print(f"Model: {model_name}: {np.mean(results[dim][model_name]):.3f} $\pm$ {np.std(results[dim][model_name]):.3f}")

## Triplet accuracy

In [ ]:
# dataset_name = 'COIL-20'
# version = "geodesic"
# models = {
#     'umap':'UMAP',
#     'tsne':'t-SNE',
#     'Basic AutoEncoder':'AE',
#     'Topological AutoEncoder':'TopoAE (Moor et.al.)',
#     'RTD AutoEncoder H1':'RTD-AE-H1',
#     'RTD AutoEncoder H2':'RTD-AE-H2'
# }

In [ ]:
def triplet_accuracy(input_data, latent_data, triplets=None):
    # calculate distance matricies
    input_data = input_data.reshape(input_data.shape[0], -1)
    input_distances = zero_out_diagonal(pdist_gpu(input_data, input_data))
    latent_data = latent_data.reshape(latent_data.shape[0], -1)
    latent_distances = zero_out_diagonal(pdist_gpu(latent_data, latent_data))
    # generate triplets
    if triplets is None:
        triplets = np.asarray(list(combinations(range(len(input_data)), r=3)))
    i_s = triplets[:, 0]
    j_s = triplets[:, 1]
    k_s = triplets[:, 2]
    acc = (np.logical_xor(
        input_distances[i_s, j_s] < input_distances[i_s, k_s], 
        latent_distances[i_s, j_s] < latent_distances[i_s, k_s]
    ) == False)
    acc = np.mean(acc.astype(np.int32))
    return acc


def avg_triplet_accuracy(input_data, latent_data, batch_size=128, n_runs=20):
    # average over batches
    accs = []
    triplets = np.asarray(list(combinations(range(min(batch_size, len(input_data))), r=3)))
    if batch_size > len(input_data):
        accs.append(triplet_accuracy(input_data, latent_data, triplets=triplets))
        return accs
    for _ in range(n_runs):
        ids = np.random.choice(np.arange(len(input_data)), size=batch_size, replace=False)
        accs.append(triplet_accuracy(input_data[ids], latent_data[ids], triplets=triplets))
    return accs

In [ ]:
input_data = np.load(f'data/{dataset_name}/prepared/train_data.npy')

for model_name in models:
    try:
        latent_data = np.load(f'data/{dataset_name}/{model_name}_output_{version}.npy')
    except FileNotFoundError:
        continue
    accs = avg_triplet_accuracy(input_data, latent_data, batch_size=150, n_runs=10)
    print(f"Model: {model_name}, triplet acc: {np.mean(accs):.3f} $\pm$ {np.std(accs):.3f}")

# RTD

Switch to ripser++ from ArGentum

In [ ]:
from src.rtd import RTDLoss
import torch

In [ ]:
n_runs = 10
batch_size = 200

loss = RTDLoss(dim=1, engine='ripser')

data = np.load(f'data/{dataset_name}/prepared/train_data.npy')
data = data.reshape(len(data), -1)

if batch_size > len(data):
    n_runs=1
    
max_dim = 1
results = defaultdict(list)

for i in tqdm(range(n_runs)):
    ids = np.random.choice(np.arange(0, len(data)), size=min(batch_size, len(data)), replace=False)
    
    x = data[ids]
    x_distances = distance_matrix(x, x)
    x_distances = x_distances/np.percentile(x_distances.flatten(), 90)
    
    for model_name in models:
        try:
            z = np.load(f'data/{dataset_name}/{model_name}_output_{version}.npy')
        except FileNotFoundError:
            try:
                z = np.load(f'data/{dataset_name}/{model_name}_output.npy')
            except FileNotFoundError:
                continue
        z = z.reshape(len(z), -1)[ids]
        z_distances = distance_matrix(z, z)
        z_distances = z_distances/np.percentile(z_distances.flatten(), 90)
        print(f'Calculating RTD for: {model_name}')
        with torch.no_grad():
            _, _, value = loss(torch.tensor(x_distances), torch.tensor(z_distances))
        results[model_name].append(value.item())

In [ ]:
names = [
    'pca', 
    'mds', 
    'tsne', 
    'umap', 
    'Basic AutoEncoder', 
    'pacmap', 
    'ivis', 
    'phate', 
    'Topological AutoEncoder', 
    'RTD AutoEncoder H1'
]
for model_name in names:
    if model_name in results:
        print(f"{model_name}: {np.mean(results[model_name]):.2f} $\pm$ {np.std(results[model_name]):.2f}")

# Tripet acc. between cluster centers

In [ ]:
def get_cluster_distances(data, labels):
    clusters = []
    if len(data.shape) > 2:
        data = data.reshape(data.shape[0], -1)
    for l in np.sort(np.unique(labels)):
        clusters.append(np.mean(data[labels == l], axis=0))
    clusters = np.asarray(clusters)
    return distance_matrix(clusters, clusters)

In [ ]:
data = np.load(f'data/{dataset_name}/prepared/train_data.npy')
labels = np.load(f'data/{dataset_name}/prepared/train_labels.npy')
original_distances = get_cluster_distances(data, labels)

In [ ]:
def triplet_accuracy_between_clusters(original_distances, latent_distances):
    triplets = np.asarray(list(combinations(range(len(original_distances)), r=3)))
    i_s = triplets[:, 0]
    j_s = triplets[:, 1]
    k_s = triplets[:, 2]
    acc = (np.logical_xor(
        original_distances[i_s, j_s] < original_distances[i_s, k_s], 
        latent_distances[i_s, j_s] < latent_distances[i_s, k_s]
    ) == False)
    return acc

def triplet_accuracy_between_clusters_(original_distances, latent_distances):
    ids = range(len(original_distances))
    triplets = np.asarray(list(product(ids, ids, ids)))
    i_s = triplets[:, 0]
    j_s = triplets[:, 1]
    k_s = triplets[:, 2]
    acc = (np.logical_xor(
        original_distances[i_s, j_s] < original_distances[i_s, k_s], 
        latent_distances[i_s, j_s] < latent_distances[i_s, k_s]
    ) == False)
    return acc

In [ ]:
for model_name in models:
    try:
        latent_data = np.load(f'data/{dataset_name}/{model_name}_output_{version}.npy')
    except FileNotFoundError:
        continue
    latent_distances = get_cluster_distances(latent_data, labels)
    accs = triplet_accuracy_between_clusters_(original_distances, latent_distances)
    print(f"Model: {model_name}, triplet acc: {np.mean(accs):.3f} $\pm$ {np.std(accs):.3f}")